<a href="https://colab.research.google.com/github/rafanonato/CPS/blob/master/robo_cripto_fgv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Exemplo de treinamento simples

In [17]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
# Baixando os dados de DOGE COIN
df = pd.read_parquet('https://drive.google.com/u/0/uc?id=17c2r9qbnsxPVxaYukrp6vhTY-CQy8WZa&export=download')

#### Calculando o target (y)

In [18]:
# Calculando qual a média de close dos próximos 10min
df['forward_average'] = df[::-1]['close'].rolling(10).mean()[::-1].shift(-1)

# Target será a diferença percentual do 'forward_average' com o 'close' atual 
df['target'] = 100*(df['forward_average'] - df['close']) / df['close']

df.head(3)
# Outra possibilidade: target como a diferença entre o proximo minuto e o atual: df['diff']= -df['close'].diff(-1)

,symbol,datetime,close_time,open,high,low,close,volume,number_of_trades,forward_average,target
open_time,,,,,,,,,,,
2019-07-05 12:00:00,DOGEUSDT,2019-07-05 12:00:00,1562328059999,0.00449,0.004600,0.00376,0.004200,60726008.0,521,0.004201,0.018099
2019-07-05 12:01:00,DOGEUSDT,2019-07-05 12:01:00,1562328119999,0.00420,0.004387,0.00420,0.004300,84307704.0,561,0.004158,-3.309539
2019-07-05 12:02:00,DOGEUSDT,2019-07-05 12:02:00,1562328179999,0.00430,0.004475,0.00430,0.004475,48182744.0,291,0.004099,-8.400496


#### Calculando as features (x)

In [5]:
# Toda a parte de criação de features está no arquivo simple_robot.py. Aqui apenas chamamos a função. Isso é útil, pois conseguimos usar a mesma função no momento de colocar o robô em produção
df = feature_eng(df)
df.head(5)

NameError: ignored

#### Separando em treino/ teste
Separando usando data. Isso é importante, pois precisamos entender se os modelos criados em um tempo passado continua sendo útil em um tempo futuro.

In [19]:
test_treshold = '2021-06-01 00:00:00'

train = df[df.index <= test_treshold]
test = df[df.index > test_treshold]

X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

# Modelo linear simples
model = sm.OLS(y_train,X_train).fit()
model.summary()

ValueError: ignored

#### Resultado do Modelo Linear

In [12]:
y_hat = model.predict(X_test)
MSE = ((y_hat - y_test)**2).mean()
MSE

NameError: ignored

In [13]:
MAE = ((y_hat - y_test).abs()).mean()
MAE

NameError: ignored

#### Referência
É sempre recomendado ter valores de referência, para saber se seu modelo é ou não melhor do que outras alternativas "naive"

Abaixo, um exemplo de resultado Naive, assumindo todos 0

In [14]:
MSE_assuming_all_zero = (y_test**2).mean()
MSE_assuming_all_zero

0.16986135624179463

In [15]:
MAE_assuming_all_zero = (y_test.abs()).mean()
MAE_assuming_all_zero

0.2633675092910254

In [16]:
# Salvando o modelo em um arquivo pickle para ser utilizado nas etapas seguintes
filename = 'model_dummy.pickle'
pickle.dump(model, open(filename, 'wb'))

NameError: ignored